In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import numpy as np
from scipy.io import wavfile as wf
# load the sample media file
# mixture = "/content/gdrive/MyDrive/Baseline Test/Copy of mixture.wav"
mixture = "/content/gdrive/MyDrive/Baseline Test/Demo Audio.wav"

# read the file into an array
sampling_rate, x = wf.read(mixture)
x=x.T
print(x.shape)
# print(x[0][1000:2000])
# print(x[1][1000:2000])

(2, 300032)


In [ ]:
# normalize the mixture
def normalize(x):
  x= np.array(x)
  x_mean = x.mean(axis=1, keepdims=True)
  x_norm = x - x_mean
  return x_norm
x_norm = normalize(x)
# print(x_norm)

In [ ]:
from numpy import linalg as LA

# whiten the data
def whiten(x):
  x_cov = np.cov(x)
  eigen = np.linalg.eigh(x_cov)
  eigen_val, eigen_vec = eigen

  D = np.diag(eigen_val)
  D_1_2 = np.sqrt(np.linalg.inv(D))

  E = eigen_vec
  E_T = E.transpose()

  x_whitened = np.matmul(E, np.matmul(D_1_2, np.matmul(E_T, x)))

  print(x_whitened.shape)

  return E, D, x_whitened

E, D, x_whitened = whiten(x_norm)




(2, 300032)


In [ ]:
def W_new(W, X):
  objective = np.tanh(np.matmul(W.T, X))
  objective_derivative = 1 -(objective ** 2)
  w_new = (X * objective).mean(axis=1) - objective_derivative.mean() * W
  w_new= w_new/np.sqrt((w_new ** 2).sum())
  return w_new


def ICA(X, epochs = 5, learning_rate = 1e-5):

  # shape of the input
  shape_ = X.shape[0]

  # create the weights matrix with the same dimensions
  weights = np.zeros((shape_, shape_), dtype=X.dtype)
  for i in np.arange(shape_):

    # initialize the matrix to random values
    w = np.random.rand(shape_)

    # iterate for the number of epochs
    for epoch in np.arange(epochs):

      # run the objective functions
      w_new = W_new(w, X)

      # train the weights
      if(i >= 1):
        weights_i = weights[:i]
        w_new= w_new - np.dot(np.dot(w_new, weights_i.T), weights_i)
      
      # calculate the improvement of the model
      improvement = np.abs((w * w_new).sum()) - 1
      improvement = np.abs(improvement)
      
      weights_updated = w_new

      # if the learning rate is achieved
      if(improvement < learning_rate):
        # done looping, learning rate achieved
        break;

    weights[i, :] = weights_updated
  
  # s = Wx
  seperated_tracks = np.dot(weights, X)

  # return s (s[0] is the source track, s[1] is the separated vocals)
  return seperated_tracks

seperated_tracks= ICA(x_whitened, epochs=100)
print(seperated_tracks.shape)
wf.write('source.wav',sampling_rate, seperated_tracks[0].astype(np.float32))
wf.write('vocals.wav', sampling_rate,seperated_tracks[1].astype(np.float32))

(2, 300032)
